In [3]:
from neo4j import GraphDatabase

In [4]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("novoadmin", "novoadmin123"))        

In [7]:
with driver.session(database="neo4j") as session:
    display(session.run("CREATE CONSTRAINT FOR (a:Article) REQUIRE a.index IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT FOR (a:Author) REQUIRE a.name IS UNIQUE").consume().counters)
    display(session.run("CREATE CONSTRAINT FOR (v:Venue) REQUIRE v.name IS UNIQUE").consume().counters)

{'_contains_updates': True, 'constraints_added': 1}

{'_contains_updates': True, 'constraints_added': 1}

{'_contains_updates': True, 'constraints_added': 1}

In [8]:
query = """
CALL apoc.periodic.iterate(
  'UNWIND ["dblp-ref-0.json", "dblp-ref-1.json", "dblp-ref-2.json", "dblp-ref-3.json"] AS file
   CALL apoc.load.json("https://github.com/mneedham/link-prediction/raw/master/data/" + file)
   YIELD value WITH value
   return value',
  'MERGE (a:Article {index:value.id})
   SET a += apoc.map.clean(value,["id","authors","references", "venue"],[0])
   WITH a, value.authors as authors, value.references AS citations, value.venue AS venue
   MERGE (v:Venue {name: venue})
   MERGE (a)-[:VENUE]->(v)
   FOREACH(author in authors | 
     MERGE (b:Author{name:author})
     MERGE (a)-[:AUTHOR]->(b))
   FOREACH(citation in citations | 
     MERGE (cited:Article {index:citation})
     MERGE (a)-[:CITED]->(cited))', 
   {batchSize: 100, maxIterations: 100});
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
    for row in result:
        print(row)

<Record batches=520 total=51956 timeTaken=21 committedOperations=51956 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 520, 'errors': {}, 'committed': 520, 'failed': 0} operations={'total': 51956, 'errors': {}, 'committed': 51956, 'failed': 0} wasTerminated=False failedParams={} updateStatistics={'relationshipsDeleted': 0, 'relationshipsCreated': 482439, 'nodesDeleted': 0, 'nodesCreated': 264616, 'labelsRemoved': 0, 'labelsAdded': 264616, 'propertiesSet': 450338}>


In [10]:
query = """
MATCH (a:Article) 
WHERE a.title IS NULL
DETACH DELETE a
"""

with driver.session(database="neo4j") as session:
    result = session.run(query)
    print(result.consume().counters)

{'_contains_updates': True, 'nodes_deleted': 132357, 'relationships_deleted': 261202}
